In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from dm_control import suite

# Importe as classes definidas do seu world model.
# Certifique-se de que o arquivo contendo essas classes (por exemplo, world_model.py)
# esteja no seu path ou no mesmo diretório.
from models import RSSM, ConvEncoder, ConvDecoder

def random_policy(action_spec):
    """Gera uma ação aleatória conforme as especificações do ambiente."""
    return np.random.uniform(low=action_spec.minimum, high=action_spec.maximum, size=action_spec.shape)

def collect_episode(env, max_steps=1000):
    """Coleta um episódio executando uma política aleatória no ambiente."""
    time_limit = 10  # limite de tempo em segundos (exemplo)
    obs = env.reset()
    images, actions = [], []
    current_time = 0.0
    while current_time < time_limit:
        # Usamos a imagem do pixel; ajuste se a chave for diferente.
        if 'pixels' in obs:
            img = obs['pixels']
        else:
            # Se a observação for um dicionário com 'image'
            img = obs['image']
        images.append(img)
        
        # Ação aleatória
        action = random_policy(env.action_spec())
        actions.append(action)
        
        timestep = env.step(action)
        obs = timestep.observation
        current_time = env.physics.data.time
        if timestep.last():
            break

    images = np.array(images)  # (T, H, W, C)
    actions = np.array(actions)  # (T, action_dim)
    return images, actions

def main():
    # Carrega o ambiente Cartpole Swing-Up.
    env = suite.load(domain_name="cartpole", task_name="swingup")
    
    # Define dispositivo de execução.
    device = torch.device("mps" if torch.mps.is_available() else "cpu")
    
    # Inicializa os modelos.
    rssm = RSSM().to(device)
    encoder = ConvEncoder().to(device)
    decoder = ConvDecoder().to(device)
    
    # Coleta um episódio.
    images_np, actions_np = collect_episode(env, max_steps=1000)
    print(f"Coletado episódio com {len(images_np)} frames.")
    
    # Pré-processa as imagens:
    # - Converte para tensor
    # - Normaliza para [0,1] e converte de (H, W, C) para (C, H, W)
    images = torch.tensor(images_np, dtype=torch.float32) / 255.0
    images = images.permute(0, 3, 1, 2)  # (T, C, H, W)
    
    # Converte as ações para tensor.
    actions = torch.tensor(actions_np, dtype=torch.float32)
    
    # Adiciona uma dimensão de batch (batch = 1).
    images = images.unsqueeze(0)  # (1, T, C, H, W)
    actions = actions.unsqueeze(0)  # (1, T, action_dim)
    
    # Codifica as imagens usando o ConvEncoder.
    # Reorganiza para processar todos os frames de uma vez.
    B, T, C, H, W = images.shape
    images_flat = images.view(B * T, C, H, W)
    embed_flat = encoder({'image': images_flat})
    embed = embed_flat.view(B, T, -1)  # (1, T, embed_dim)
    
    # Executa o método observe do RSSM para obter os estados latentes (post e prior).
    post, prior = rssm.observe(embed, actions)
    
    # Gera uma trajetória imaginada (apenas usando ações) a partir do estado inicial.
    imagine_prior = rssm.imagine(actions)
    
    # Exibe as formas das distribuições latentes.
    print("Forma da variável 'stoch' posterior:", post['stoch'].shape)
    print("Forma da variável 'stoch' imaginada:", imagine_prior['stoch'].shape)
    
    # Reconstrói a primeira imagem a partir do primeiro estado posterior.
    # Obtemos os features concatenando stoch e deter.
    first_state = {k: post[k][:, 0] for k in post}
    features = rssm.get_feat(first_state)
    recon_dist = decoder(features)
    recon_img = recon_dist.mean  # (batch, C, H, W)
    
    # Exibe as formas das imagens.
    print("Forma da imagem original:", images[0, 0].shape)
    print("Forma da imagem reconstruída:", recon_img.shape)
    
    # Converte para numpy e reorganiza para (H, W, C) para exibição.
    orig_img = images[0, 0].permute(1, 2, 0).cpu().numpy()
    recon_img_np = recon_img[0].permute(1, 2, 0).detach().cpu().numpy()
    
    # Plota a imagem original e a reconstruída.
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(orig_img)
    plt.title("Original")
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(recon_img_np)
    plt.title("Reconstruída")
    plt.axis('off')
    
    plt.show()

if __name__ == '__main__':
    main()


TypeError: tuple indices must be integers or slices, not str

In [23]:
def compute_v(rewards, values, tau, k, gamma=0.99, lamb=0.95, t=0):
    """
    v_lambda : (1,B)
    """
    horizon, B = rewards.shape
    h = min(tau + k, t + horizon - 1)
    
    v = torch.zeros(1, B, dtype=rewards.dtype, device=rewards.device)
    
    for n in range(tau, h - 1):
        v = (gamma ** (n - tau)) * rewards[n] + (gamma ** (h - tau) * values[h]) + v
    
    return v

def compute_v_lambda(rewards, values, tau, gamma=0.99, lamb=0.95):
    horizon, B = rewards.shape
    v_lambda = torch.zeros(1, B, dtype=rewards.dtype, device=rewards.device)
    for n in range(1, horizon - 1):
        v1 = (lamb ** (n - 1) * compute_v(rewards, values, tau, n))
        v2 = (lamb ** (horizon - 1))
        v3 = compute_v(rewards, values, tau, horizon)
        v_lambda = v1 + v2 * v3 + v_lambda
    v_lambda = (1 - lamb) * v_lambda
    
    print(v1,v_lambda)
    return v_lambda

In [24]:
from auxiliares import training_device
import torch
device = training_device()

# Definindo horizon e batch size
horizon = 4  # número de timesteps
B = 3        # batch size

# Cria um tensor de recompensas com valores de exemplo (dimensão: horizon x B)
rewards = torch.tensor([
    [7.5, 2.3, 3.0],
    [0.5, 1.0, 1.5],
    [0.2, 0.4, 0.6],
    [0.1, 0.2, 0.3]
], device=device)

# Cria um tensor de valores com valores de exemplo (dimensão: horizon x B)
values = torch.tensor([
    [10.0, 20.0, 30.0],
    [11.0, 21.0, 31.0],
    [12.0, 22.0, 32.0],
    [13.0, 23.0, 33.0]
], device=device)

# Parâmetros para a função compute_v
tau = 0
k = 2
gamma = 0.99

# Chama a função e imprime o resultado
v = compute_v_lambda(rewards, values, tau, k)
print("Valor computado v:", v)

mps
tensor([[18.2981, 22.6691, 32.6450]], device='mps:0') tensor([[3.7633, 5.2423, 7.5074]], device='mps:0')
Valor computado v: tensor([[3.7633, 5.2423, 7.5074]], device='mps:0')
